# Determining the Fair Spread of a CDS Tranche

An analysis of pricing a CDS Index using its intrinsic value

In [1]:
import numpy as np
import matplotlib.pyplot as plt

In [2]:
from financepy.utils import *
from financepy.products.rates import *
from financepy.products.credit import *

In [3]:
value_dt = Date(2, 8, 2007)
settle_dt = value_dt.add_weekdays(1)

## Build Ibor Curve

In [4]:
dc_type = DayCountTypes.THIRTY_E_360_ISDA
depos = []
depo = IborDeposit(value_dt, "1D", 0.0500, dc_type); depos.append(depo)
depo = IborDeposit(settle_dt, "1D", 0.0500, dc_type); depos.append(depo)

fixed_freq = FrequencyTypes.SEMI_ANNUAL
swap_type = SwapTypes.PAY
swap1 = IborSwap(settle_dt,"1Y",swap_type,0.0502,fixed_freq,dc_type)
swap2 = IborSwap(settle_dt,"2Y",swap_type,0.0502,fixed_freq,dc_type)
swap3 = IborSwap(settle_dt,"3Y",swap_type,0.0501,fixed_freq,dc_type)
swap4 = IborSwap(settle_dt,"4Y",swap_type,0.0502,fixed_freq,dc_type)
swap5 = IborSwap(settle_dt,"5Y",swap_type,0.0501,fixed_freq,dc_type)
swaps = [swap1,swap2,swap3,swap4,swap5]

libor_curve = IborSingleCurve(value_dt, depos, [], swaps)

We treat an index as a CDS contract with a flat CDS curve at the CDS index spread for the same maturity

## Create the Underlying CDS Index Portfolio

In [5]:
step_in_dt = value_dt.add_weekdays(1)

In [6]:
maturity_3yr = value_dt.next_cds_date(36)
maturity_5yr = value_dt.next_cds_date(60)
maturity_7yr = value_dt.next_cds_date(84)
maturity_10yr = value_dt.next_cds_date(120)

### Heterogeneous Curves

In [7]:
# Move into the local folder of notebook
%pwd
%cd notebooks/products/credit/
%pwd


[WinError 3] The system cannot find the path specified: 'notebooks/products/credit/'
C:\Users\Dominic\Dropbox\Desktop\RESEARCH_DB\FinancePy\Code\financepy-git\notebooks\products\credit


'C:\\Users\\Dominic\\Dropbox\\Desktop\\RESEARCH_DB\\FinancePy\\Code\\financepy-git\\notebooks\\products\\credit'

In [8]:
f = open('.//data//CDX_NA_IG_S7_SPREADS.csv', 'r')
data = f.readlines()
heteroIssuerCurves = []

num_credits = len(data) - 1  # The file has a header

for row in data[1:]:
    splitRow = row.split(",")
    spd3Y = float(splitRow[1]) / 10000.0
    spd5Y = float(splitRow[2]) / 10000.0
    spd7Y = float(splitRow[3]) / 10000.0
    spd10Y = float(splitRow[4]) / 10000.0
    recovery_rate = float(splitRow[5])
    cds3Y = CDS(step_in_dt, maturity_3yr, spd3Y)
    cds5Y = CDS(step_in_dt, maturity_5yr, spd5Y)
    cds7Y = CDS(step_in_dt, maturity_7yr, spd7Y)
    cds10Y = CDS(step_in_dt, maturity_10yr, spd10Y)
    cds_contracts = [cds3Y, cds5Y, cds7Y, cds10Y]
    issuer_curve = CDSCurve(value_dt, cds_contracts, libor_curve, recovery_rate)
    heteroIssuerCurves.append(issuer_curve)

### Homogeneous Curves 

Calculate the average spread of the heterogeneous portfolio

In [9]:
homoIssuerCurves = []
num_credits = 125
recovery_rate = 0.40

In [10]:
cdsIndex = CDSIndexPortfolio()

In [11]:
spd3Y = cdsIndex.intrinsic_spread(value_dt, step_in_dt, maturity_3yr, heteroIssuerCurves)
spd5Y = cdsIndex.intrinsic_spread(value_dt, step_in_dt, maturity_5yr, heteroIssuerCurves)
spd7Y = cdsIndex.intrinsic_spread(value_dt, step_in_dt, maturity_7yr, heteroIssuerCurves)
spd10Y = cdsIndex.intrinsic_spread(value_dt, step_in_dt, maturity_10yr, heteroIssuerCurves)

In [12]:
print("Homogeneous curve 3Y:", spd3Y*10000)
print("Homogeneous curve 5Y:", spd5Y*10000)
print("Homogeneous curve 7Y:", spd7Y*10000)
print("Homogeneous curve 10Y:", spd10Y*10000)

Homogeneous curve 3Y: 19.67879398295557
Homogeneous curve 5Y: 35.539142389477774
Homogeneous curve 7Y: 49.01183816666462
Homogeneous curve 10Y: 61.413769315177525


In [13]:
for row in range(0,num_credits):
    cds3Y = CDS(step_in_dt, maturity_3yr, spd3Y)
    cds5Y = CDS(step_in_dt, maturity_5yr, spd5Y)
    cds7Y = CDS(step_in_dt, maturity_7yr, spd7Y)
    cds10Y = CDS(step_in_dt, maturity_10yr, spd10Y)
    cds_contracts = [cds3Y, cds5Y, cds7Y, cds10Y]
    issuer_curve = CDSCurve(value_dt, cds_contracts, libor_curve, recovery_rate)
    homoIssuerCurves.append(issuer_curve)

## Define the Tranches

In [14]:
trancheMaturity = maturity_5yr
tranche1 = CDSTranche(value_dt, trancheMaturity, 0.00, 0.03)
tranche2 = CDSTranche(value_dt, trancheMaturity, 0.03, 0.06)
tranche3 = CDSTranche(value_dt, trancheMaturity, 0.06, 0.09)
tranche4 = CDSTranche(value_dt, trancheMaturity, 0.09, 0.12)
tranche5 = CDSTranche(value_dt, trancheMaturity, 0.12, 0.22)
tranche6 = CDSTranche(value_dt, trancheMaturity, 0.22, 0.60)
tranche7 = CDSTranche(value_dt, trancheMaturity, 0.00, 0.60)
tranche8 = CDSTranche(value_dt, trancheMaturity, 0.00, 1.00)

In [15]:
tranches = [tranche1, tranche2, tranche3, tranche4, tranche5, tranche6, tranche7, tranche8]

In [16]:
corr1 = 0.30
corr2 = 0.30
upfront = 0.0
spd = 0.0

## Homogeneous Portfolio Results

In [17]:
print("%50s %5s %9s %9s %12s"% ("Method", "NumPts", "k_1", "k_2", "SPD(BPS)"))
for tranche in tranches:
    for method in FinLossDistributionBuilder:
        for num_points in [50]:
            v = tranche.value_bc(value_dt,homoIssuerCurves,upfront,spd,corr1,corr2,num_points,method)
            print("%50s %5d %9.5f %9.5f %12.6f"% (method, num_points, tranche.k1*100, tranche.k2*100, v[3] * 10000))
    print("=============================================================================================")

                                            Method NumPts       k_1       k_2     SPD(BPS)
              FinLossDistributionBuilder.RECURSION    50   0.00000   3.00000   875.088146
      FinLossDistributionBuilder.ADJUSTED_BINOMIAL    50   0.00000   3.00000   875.088146
               FinLossDistributionBuilder.GAUSSIAN    50   0.00000   3.00000   908.571411


                    FinLossDistributionBuilder.LHP    50   0.00000   3.00000   914.366815
              FinLossDistributionBuilder.RECURSION    50   3.00000   6.00000   239.580083
      FinLossDistributionBuilder.ADJUSTED_BINOMIAL    50   3.00000   6.00000   239.580083
               FinLossDistributionBuilder.GAUSSIAN    50   3.00000   6.00000   239.269213
                    FinLossDistributionBuilder.LHP    50   3.00000   6.00000   226.808804
              FinLossDistributionBuilder.RECURSION    50   6.00000   9.00000   102.124297
      FinLossDistributionBuilder.ADJUSTED_BINOMIAL    50   6.00000   9.00000   102.124297
               FinLossDistributionBuilder.GAUSSIAN    50   6.00000   9.00000   101.792518
                    FinLossDistributionBuilder.LHP    50   6.00000   9.00000    94.918261
              FinLossDistributionBuilder.RECURSION    50   9.00000  12.00000    49.210550
      FinLossDistributionBuilder.ADJUSTED_BINOMIAL    50   9.00000  12.00000    49.210550
          

               FinLossDistributionBuilder.GAUSSIAN    50  22.00000  60.00000     0.488422
                    FinLossDistributionBuilder.LHP    50  22.00000  60.00000     0.418217
              FinLossDistributionBuilder.RECURSION    50   0.00000  60.00000    59.215546
      FinLossDistributionBuilder.ADJUSTED_BINOMIAL    50   0.00000  60.00000    59.215546
               FinLossDistributionBuilder.GAUSSIAN    50   0.00000  60.00000    60.178480
                    FinLossDistributionBuilder.LHP    50   0.00000  60.00000    59.215539
              FinLossDistributionBuilder.RECURSION    50   0.00000 100.00000    35.376821
      FinLossDistributionBuilder.ADJUSTED_BINOMIAL    50   0.00000 100.00000    35.376821
               FinLossDistributionBuilder.GAUSSIAN    50   0.00000 100.00000    35.945117
                    FinLossDistributionBuilder.LHP    50   0.00000 100.00000    35.376814


## Heterogeneous Portfolio Results

In [18]:
print("%50s %5s %9s %9s %12s"% ("Method", "NumPts", "k_1", "k_2", "SPD(BPS)"))

for tranche in tranches:
    for method in FinLossDistributionBuilder:
        for num_points in [50]:
            v = tranche.value_bc(value_dt,heteroIssuerCurves,upfront,spd,corr1,corr2,num_points,method)
            print("%50s %5d  %9.5f %9.5f %12.6f"% (method, num_points, tranche.k1*100, tranche.k2*100, v[3] * 10000))
    print("=============================================================================================")

                                            Method NumPts       k_1       k_2     SPD(BPS)
              FinLossDistributionBuilder.RECURSION    50    0.00000   3.00000   949.541331
      FinLossDistributionBuilder.ADJUSTED_BINOMIAL    50    0.00000   3.00000   949.473315
               FinLossDistributionBuilder.GAUSSIAN    50    0.00000   3.00000   983.192109
                    FinLossDistributionBuilder.LHP    50    0.00000   3.00000   914.736913
              FinLossDistributionBuilder.RECURSION    50    3.00000   6.00000   230.886128
      FinLossDistributionBuilder.ADJUSTED_BINOMIAL    50    3.00000   6.00000   230.978141
               FinLossDistributionBuilder.GAUSSIAN    50    3.00000   6.00000   230.441917
                    FinLossDistributionBuilder.LHP    50    3.00000   6.00000   226.808075
              FinLossDistributionBuilder.RECURSION    50    6.00000   9.00000    87.195509
      FinLossDistributionBuilder.ADJUSTED_BINOMIAL    50    6.00000   9.00000    87.253318

               FinLossDistributionBuilder.GAUSSIAN    50    9.00000  12.00000    37.313566
                    FinLossDistributionBuilder.LHP    50    9.00000  12.00000    45.123579
              FinLossDistributionBuilder.RECURSION    50   12.00000  22.00000     8.938853
      FinLossDistributionBuilder.ADJUSTED_BINOMIAL    50   12.00000  22.00000     8.955035


               FinLossDistributionBuilder.GAUSSIAN    50   12.00000  22.00000     8.925328
                    FinLossDistributionBuilder.LHP    50   12.00000  22.00000    12.626359
              FinLossDistributionBuilder.RECURSION    50   22.00000  60.00000     0.212516
      FinLossDistributionBuilder.ADJUSTED_BINOMIAL    50   22.00000  60.00000     0.212590
               FinLossDistributionBuilder.GAUSSIAN    50   22.00000  60.00000     0.212040
                    FinLossDistributionBuilder.LHP    50   22.00000  60.00000     0.418026
              FinLossDistributionBuilder.RECURSION    50    0.00000  60.00000    59.215520
      FinLossDistributionBuilder.ADJUSTED_BINOMIAL    50    0.00000  60.00000    59.215520
               FinLossDistributionBuilder.GAUSSIAN    50    0.00000  60.00000    60.119200
                    FinLossDistributionBuilder.LHP    50    0.00000  60.00000    59.215495
              FinLossDistributionBuilder.RECURSION    50    0.00000 100.00000    35.376468

Copyright (c) 2020 Dominic O'Kane